---

# **Importing of Libraries**

- In this section, we have imported all the libraries that we will used to scrap snapdeal products.

---

In [1]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

---

# **Shopclues Scraper**

- In this section, we will scrap the items as highlighted in the red box in the following image.

![img](./images/shopclues.PNG?raw=true)

- We will use the following user defined function to extract products from shopclues website.

---

In [6]:
def main(search, num_of_items):
    """Executes the main program."""

    # Initialize the Chrome driver
    driver = webdriver.Chrome("chromedriver")
    driver.get('https://www.shopclues.com/')

    # Search the products
    driver.find_element_by_xpath('//*[@id="autocomplete"]').send_keys(Keys.CONTROL + "a")
    driver.find_element_by_xpath('//*[@id="autocomplete"]').send_keys(search)
    driver.find_element_by_xpath('//*[@id="autocomplete"]').send_keys(Keys.RETURN)

    time.sleep(5)

    current_scroll_position = 0
    new_height = 0.8
    sum = 0
    speed = 8
    data_items = num_of_items
    records = list()

    while sum <= data_items:

        # Screen scroller to get similar items
        while current_scroll_position <= new_height:

            current_scroll_position += speed
            driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
            new_height = driver.execute_script("return document.body.scrollHeight")
            time.sleep(0.015)

        # Parse items from start to the scrolled location of the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.findAll('div', attrs={'class': 'row'})
        for result in results:

            items = result.findAll('div', {'class': 'search_blocks'})
            for item in items:
                try:
                    URL = item.find('a', {'target': '_blank'})['href'].replace('//', '')
                except AttributeError:
                    URL = ''

                try:
                    title = item.find('h2').text
                except AttributeError:
                    title = ''

                try:
                    striked_price = item.find('div', {'class': 'old_prices'}).find('span', {'class': 'p_price'}).text.strip()
                except AttributeError:
                    striked_price = ''

                try:
                    price = item.find('div', {'class': 'ori_price'}).find('span', {'class': 'p_price'}).text.strip()
                except AttributeError:
                    price = ''

                try:
                    price_off = item.find('div', {'class': 'ori_price'}).find('span', {'class': 'prd_discount'}).text.strip()
                except AttributeError:
                    price_off = ''

                try:
                    free_shipping = item.find('div', {'class': 'shipping'}).text.strip()
                except AttributeError:
                    free_shipping = ''

                tuple_record = (URL, title, price, striked_price, price_off, free_shipping)
                records.append(tuple_record)

            sum = sum + len(items)

        time.sleep(2)
        
        if sum <= data_items:
            button = driver.find_element_by_xpath('//*[@id="moreProduct"]')
            driver.execute_script("arguments[0].click();", button)

    driver.close()

    # # Saving file to a CSV
    data = pd.DataFrame(data=records, columns=['URL', 'Title', 'Price', 'StrikedPrice', 'PriceOff', 'RatingsCount'])
    data.to_csv('results.csv', index=False)
    print('Records saved to the file!, Displaying top 5 rows...')
    return data.head()


In [7]:
# Search and get the data associated with the product
search = input("Enter the Product:")
main(search=search, num_of_items=50)

Records saved to the file!, Displaying top 5 rows...


,URL,Title,Price,StrikedPrice,PriceOff,RatingsCount
0,www.shopclues.com/i-kall-k900-smartphone-5.5-i...,I Kall K900 5.5 inches(13.97 cm) Display 4GB R...,"₹5,079","₹5,999",36% Off,Free Shipping
1,www.shopclues.com/i-kall-k260-5.5-inch-display...,I KALL K260 5.5 inches(13.97 cm) 2 GB 16 GB Du...,"₹4,299","₹4,599",28% Off,Free Shipping
2,www.shopclues.com/samsung-galaxy-f02s-3gb-32gb...,Samsung Galaxy F02s (3gb / 32gb) Dark Blue,"₹9,299","₹10,499",11% Off,
3,www.shopclues.com/ikall-k5-smart-phone-4g-volt...,IKall K5 Smart Phone 4G VoLTE 2 GB 16 GB with...,"₹4,199","₹4,499",33% Off,Free Shipping
4,www.shopclues.com/ikall-k4-smart-phone-4g-volt...,IKall K4 2 GB RAM 16 GB ROM Smart Phone With B...,"₹3,899","₹4,199",38% Off,Free Shipping
